# Data Science Conference in New York

Introduction

The Hut Group is a company based in New York is organizing event for 5 days for a group of Data Scientists from all over the world.The company has to put a good program, including a hotel of residence, a hall for meetings, places of landscape to visit, stores for shopping, restaurants,carparks and cafes. So the company’s purpose is to make a list of places of landscape in Manchester, including the nearest restaurants, cafes, and shopping stores for each place.

Data Description
The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area.

Table of contents

1.Import Libraries

2.Define Foursquare Credentials

3.Define the city and get its latitude & longitude

4.Search for Hotels & clean dataframe

5.Search for Parking & clean dataframe

6.Search for Restaurants & clean dataframe

7.Search for Shopping Stores & clean dataframe

8.Generate map to visualize hotels, shopping stores,Parking and Restaurant and how they cluster together

##  Import Libraries

In [1]:
pip install folium


The following command must be run outside of the IPython shell:

    $ pip install folium

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [3]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('packages imported')

packages imported


## Define Foursquare Credentials

In [4]:
# @hidden_cell

## Define the city and get its latitude & longitude

In [5]:
# define the city and get its latitude & longitude 
city = 'New York'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


## Hotel Search 

In [6]:
# Search hotels and get hotel dataset. 
search_query = 'Hotel'
radius = 700

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results = requests.get(url).json()

venues = results['response']['venues']

# tranform venues into a dataframe
data = json_normalize(venues)
data.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,56d8c0f8498edb854f926e6a,123 Nassau St,US,New York,United States,Beekman St,182,"[123 Nassau St (Beekman St), New York, NY 1003...","[{'label': 'display', 'lat': 40.7111725, 'lng'...",40.711173,-74.006702,10038,NY,The Beekman – A Thompson Hotel,v-1581088597,NaN
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,3fd66200f964a520bbe61ee3,2 Avenue of the Americas,US,New York,United States,btwn Walker & White St,740,[2 Avenue of the Americas (btwn Walker & White...,"[{'label': 'display', 'lat': 40.71934055853672...",40.719341,-74.005044,10013,NY,The Roxy Hotel,v-1581088597,72165046
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,57c640ad498e74977f98372f,27 Barclay St,US,New York,United States,Broadway,284,"[27 Barclay St (Broadway), New York, NY 10007,...","[{'label': 'display', 'lat': 40.71261246501983...",40.712612,-74.009380,10007,NY,Four Seasons Hotel New York Downtown,v-1581088597,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,49efcc88f964a52006691fe3,85 W Broadway,US,New York,United States,Chambers St,379,"[85 W Broadway (Chambers St), New York, NY 100...","[{'label': 'display', 'lat': 40.7151439, 'lng'...",40.715144,-74.009183,10007,NY,Smyth Hotel,v-1581088597,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,5648ead638fa50411e04d842,396 Broadway,US,New York,United States,Walker Street,706,"[396 Broadway (Walker Street), New York, NY 10...","[{'label': 'display', 'lat': 40.71848123880058...",40.718481,-74.002469,10013,NY,Walker Hotel Tribeca,v-1581088597,NaN


In [7]:
# Data cleaning
clean_columns = ['name', 'categories'] + [col for col in data.columns if col.startswith('location.')]+ ['id']
clean_data = data.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_data['categories'] = clean_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_data.columns = [column.split('.')[-1] for column in clean_data.columns]

clean_data.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Beekman – A Thompson Hotel,Hotel,123 Nassau St,US,New York,United States,Beekman St,182,"[123 Nassau St (Beekman St), New York, NY 1003...","[{'label': 'display', 'lat': 40.7111725, 'lng'...",40.711173,-74.006702,10038,NY,56d8c0f8498edb854f926e6a
1,The Roxy Hotel,Hotel,2 Avenue of the Americas,US,New York,United States,btwn Walker & White St,740,[2 Avenue of the Americas (btwn Walker & White...,"[{'label': 'display', 'lat': 40.71934055853672...",40.719341,-74.005044,10013,NY,3fd66200f964a520bbe61ee3
2,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,US,New York,United States,Broadway,284,"[27 Barclay St (Broadway), New York, NY 10007,...","[{'label': 'display', 'lat': 40.71261246501983...",40.712612,-74.009380,10007,NY,57c640ad498e74977f98372f
3,Smyth Hotel,Hotel,85 W Broadway,US,New York,United States,Chambers St,379,"[85 W Broadway (Chambers St), New York, NY 100...","[{'label': 'display', 'lat': 40.7151439, 'lng'...",40.715144,-74.009183,10007,NY,49efcc88f964a52006691fe3
4,Walker Hotel Tribeca,Hotel,396 Broadway,US,New York,United States,Walker Street,706,"[396 Broadway (Walker Street), New York, NY 10...","[{'label': 'display', 'lat': 40.71848123880058...",40.718481,-74.002469,10013,NY,5648ead638fa50411e04d842


In [8]:
# delete unnecessary columns
clean_data= clean_data.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','id'], axis=1)

clean_data.head()

,name,categories,address,lat,lng,postalCode,state
0,The Beekman – A Thompson Hotel,Hotel,123 Nassau St,40.711173,-74.006702,10038,NY
1,The Roxy Hotel,Hotel,2 Avenue of the Americas,40.719341,-74.005044,10013,NY
2,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,10007,NY
3,Smyth Hotel,Hotel,85 W Broadway,40.715144,-74.009183,10007,NY
4,Walker Hotel Tribeca,Hotel,396 Broadway,40.718481,-74.002469,10013,NY


In [9]:
# delete rows with none values (missing values)
clean_data = clean_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_data.head()

,name,categories,address,lat,lng,postalCode,state
0,The Beekman – A Thompson Hotel,Hotel,123 Nassau St,40.711173,-74.006702,10038,NY
1,The Roxy Hotel,Hotel,2 Avenue of the Americas,40.719341,-74.005044,10013,NY
2,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,10007,NY
3,Smyth Hotel,Hotel,85 W Broadway,40.715144,-74.009183,10007,NY
4,Walker Hotel Tribeca,Hotel,396 Broadway,40.718481,-74.002469,10013,NY


In [10]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_data= clean_data.loc[clean_data['categories'].isin(array)]
hotel_data.head()

,name,categories,address,lat,lng,postalCode,state
0,The Beekman – A Thompson Hotel,Hotel,123 Nassau St,40.711173,-74.006702,10038,NY
1,The Roxy Hotel,Hotel,2 Avenue of the Americas,40.719341,-74.005044,10013,NY
2,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,10007,NY
3,Smyth Hotel,Hotel,85 W Broadway,40.715144,-74.009183,10007,NY
4,Walker Hotel Tribeca,Hotel,396 Broadway,40.718481,-74.002469,10013,NY


In [11]:
# delete rows which has duplicate hotel's name
print('Hotel data size:',hotel_data.shape)
hotel_data = hotel_data.drop_duplicates(subset='name', keep="first")
print('Hotel data size:',hotel_data.shape)

Hotel data size: (14, 7)
Hotel data size: (14, 7)


In [12]:
# choose the hotel which has the same postalCode with the event space
df_hotel = hotel_data[hotel_data.postalCode == '10007']
df_hotel

,name,categories,address,lat,lng,postalCode,state
2,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,40.712612,-74.009380,10007,NY
3,Smyth Hotel,Hotel,85 W Broadway,40.715144,-74.009183,10007,NY
9,Cosmopolitan Hotel - TriBeCa,Hotel,95 W Broadway,40.715622,-74.009115,10007,NY
13,The Frederick Hotel,Hotel,95 W Broadway,40.715675,-74.009047,10007,NY


## Park Search

In [13]:
# search for Parks
search_query = 'Park'
radius = 100000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
park_results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = park_results['response']['venues']

# tranform venues into a dataframe
park_data = json_normalize(venues)
park_data.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520d8f11ee3,17 Park Row,...,69,"[17 Park Row (btwn Broadway & Centre St), New ...",NaN,40.712415,-74.006724,NaN,10038,NY,City Hall Park,v-1581088598
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520def11ee3,btwn Broadway & Union Sq E,...,2813,[btwn Broadway & Union Sq E (btwn E 14th & E 1...,NaN,40.735170,-73.990667,NaN,10003,NY,Union Square Park,v-1581088598
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,42daf100f964a52035261fe3,Battery Pl,...,1342,"[Battery Pl (at State St), New York, NY 10004,...","[{'label': 'display', 'lat': 40.70321688525361...",40.703217,-74.015795,NaN,10004,NY,Battery Park,v-1581088598
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,430d0a00f964a5203e271fe3,Main St,...,1415,"[Main St (Plymouth St), Brooklyn, NY 11201, Un...","[{'label': 'display', 'lat': 40.702282, 'lng':...",40.702282,-73.996456,"DUMBO, Brooklyn, NY",11201,NY,Brooklyn Bridge Park,v-1581088598
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,412d2800f964a520df0c1fe3,59th St to 110th St,...,8668,[59th St to 110th St (5th Ave to Central Park ...,"[{'label': 'display', 'lat': 40.78408342593807...",40.784083,-73.964853,NaN,10028,NY,Central Park,v-1581088598


In [14]:
# keep only columns that include venue name, and anything that is associated with location
park_clean_columns = ['name', 'categories'] + [col for col in park_data.columns if col.startswith('location.')]+ ['id']
clean_park_data = park_data.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_data['categories'] = clean_park_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_data.columns = [column.split('.')[-1] for column in clean_park_data.columns]

clean_park_data.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,City Hall Park,Park,17 Park Row,US,New York,United States,btwn Broadway & Centre St,69,"[17 Park Row (btwn Broadway & Centre St), New ...",NaN,40.712415,-74.006724,NaN,10038,NY,3fd66200f964a520d8f11ee3
1,Union Square Park,Park,btwn Broadway & Union Sq E,US,New York,United States,btwn E 14th & E 17th St,2813,[btwn Broadway & Union Sq E (btwn E 14th & E 1...,NaN,40.735170,-73.990667,NaN,10003,NY,3fd66200f964a520def11ee3
2,Battery Park,Park,Battery Pl,US,New York,United States,at State St,1342,"[Battery Pl (at State St), New York, NY 10004,...","[{'label': 'display', 'lat': 40.70321688525361...",40.703217,-74.015795,NaN,10004,NY,42daf100f964a52035261fe3
3,Brooklyn Bridge Park,Park,Main St,US,Brooklyn,United States,Plymouth St,1415,"[Main St (Plymouth St), Brooklyn, NY 11201, Un...","[{'label': 'display', 'lat': 40.702282, 'lng':...",40.702282,-73.996456,"DUMBO, Brooklyn, NY",11201,NY,430d0a00f964a5203e271fe3
4,Central Park,Park,59th St to 110th St,US,New York,United States,5th Ave to Central Park West,8668,[59th St to 110th St (5th Ave to Central Park ...,"[{'label': 'display', 'lat': 40.78408342593807...",40.784083,-73.964853,NaN,10028,NY,412d2800f964a520df0c1fe3


In [15]:
# delete unnecessary columns
clean_park_data= clean_park_data.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)

clean_park_data.head()

,name,categories,address,lat,lng,neighborhood,postalCode,state
0,City Hall Park,Park,17 Park Row,40.712415,-74.006724,NaN,10038,NY
1,Union Square Park,Park,btwn Broadway & Union Sq E,40.735170,-73.990667,NaN,10003,NY
2,Battery Park,Park,Battery Pl,40.703217,-74.015795,NaN,10004,NY
3,Brooklyn Bridge Park,Park,Main St,40.702282,-73.996456,"DUMBO, Brooklyn, NY",11201,NY
4,Central Park,Park,59th St to 110th St,40.784083,-73.964853,NaN,10028,NY


In [16]:
# delete rows with none values (missing values)
clean_park_data = clean_park_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_data.head()

,name,categories,address,lat,lng,neighborhood,postalCode,state
3,Brooklyn Bridge Park,Park,Main St,40.702282,-73.996456,"DUMBO, Brooklyn, NY",11201,NY
9,Bryant Park,Park,E 42nd St,40.753621,-73.983265,Midtown,10018,NY
25,New York Public Library - Battery Park City,Library,175 N End Ave,40.715758,-74.015603,Battery Park City,10282,NY


In [17]:
# carparks near event 
clean_park_data[(clean_park_data.postalCode.str.contains('1')) &(clean_park_data.categories=='Park')]

,name,categories,address,lat,lng,neighborhood,postalCode,state
3,Brooklyn Bridge Park,Park,Main St,40.702282,-73.996456,"DUMBO, Brooklyn, NY",11201,NY
9,Bryant Park,Park,E 42nd St,40.753621,-73.983265,Midtown,10018,NY


## Restaurant Search 

In [18]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

Restaurant_data = requests.get(url).json()

# assign relevant part of JSON to venues
venues = Restaurant_data['response']['venues']

# tranform venues into a dataframe
Restaurant_data = json_normalize(venues)

# data cleaning 
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_data.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_data = Restaurant_data.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_data['categories'] = clean_Restaurant_data.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_data.columns = [column.split('.')[-1] for column in clean_Restaurant_data.columns]

# delete unnecessary columns
clean_Restaurant_data= clean_Restaurant_data.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)

# delete rows with none values
df_Restaurant = clean_Restaurant_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head()

,name,categories,address,lat,lng,postalCode,state
0,Jing Fong Restaurant 金豐大酒樓,Dim Sum Restaurant,20 Elizabeth St,40.715881,-73.997209,10013,NY
1,Golden Unicorn Restaurant 麒麟金閣,Dim Sum Restaurant,18 E Broadway,40.713629,-73.997230,10002,NY
2,O'Hara's Restaurant & Pub,Pub,120 Cedar St,40.709894,-74.012836,10006,NY
3,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,40.715336,-74.008881,10007,NY
4,Bo Ky Restaurant 波記潮州小食,Chinese Restaurant,80 Bayard St,40.715696,-73.998667,10013,NY



## Shopping Market Search 

In [19]:
# search for Shopping
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
sresults = requests.get(url).json()

# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)

# clean shopping data 
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]
clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Shopping Box,Miscellaneous Shop,360 Broome St,US,New York,United States,Mott,1217,"[360 Broome St (Mott), New York, NY 10013, Uni...","[{'label': 'display', 'lat': 40.720156, 'lng':...",40.720156,-73.995424,10013,NY,4c23bb79f1272d7f8ae781c5
1,Mei Bang Shopping Center,Clothing Store,NaN,US,New York,United States,NaN,765,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.713654, 'lng':...",40.713654,-73.997029,NaN,NY,5252c341498e2b30e7cde0b8
2,Mobile Shopping Fall,Event Space,NaN,US,NaN,United States,NaN,774,"[New York, United States]","[{'label': 'display', 'lat': 40.706525, 'lng':...",40.706525,-74.001860,NaN,New York,4e9d77180aaf015a05f89844
3,derek loves shopping,Office,NaN,US,NaN,United States,NaN,825,"[New York, United States]","[{'label': 'display', 'lat': 40.7066, 'lng': -...",40.706600,-74.011516,NaN,New York,4b7edcadf964a520090530e3
4,AzokaBD WOMEN'S SHOPPING,Market,19021 Midway Rd,US,Dallas,United States,NaN,902,"[19021 Midway Rd, Dallas, TX 75287, United Sta...","[{'label': 'display', 'lat': 40.71223177869945...",40.712232,-73.995345,75287,TX,59178da93abcaf1ddc0a6e3f


In [20]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)

# delete rows which its category is not Shopping Mall
#df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping = clean_Shopping_dataframe2.copy()
df_Shopping.head()

,name,categories,address,lat,lng,postalCode,state
0,Shopping Box,Miscellaneous Shop,360 Broome St,40.720156,-73.995424,10013,NY
1,Mei Bang Shopping Center,Clothing Store,NaN,40.713654,-73.997029,NaN,NY
2,Mobile Shopping Fall,Event Space,NaN,40.706525,-74.001860,NaN,New York
3,derek loves shopping,Office,NaN,40.706600,-74.011516,NaN,New York
4,AzokaBD WOMEN'S SHOPPING,Market,19021 Midway Rd,40.712232,-73.995345,75287,TX


In [21]:
# create dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, clean_park_data, df_Restaurant, df_Shopping], ignore_index=True,sort=False)

hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='University']
hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='Jewelry Store']
hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='College Residence Hall']
hotel_neighbourhood_df = hotel_neighbourhood_df[hotel_neighbourhood_df.categories!='Building']

clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Shopping Box,Miscellaneous Shop,360 Broome St,40.720156,-73.995424,10013,NY
1,Mei Bang Shopping Center,Clothing Store,NaN,40.713654,-73.997029,NaN,NY
2,Mobile Shopping Fall,Event Space,NaN,40.706525,-74.001860,NaN,New York
3,derek loves shopping,Office,NaN,40.706600,-74.011516,NaN,New York
4,AzokaBD WOMEN'S SHOPPING,Market,19021 Midway Rd,40.712232,-73.995345,75287,TX
5,Shop Gotham NYC Shopping Tours,Travel Agency,72 Spring St,40.722022,-73.997254,10012,NY
6,iBraggiotti Jewelry,Jewelry Store,21 Warren St,40.714107,-74.007786,10007,NY
7,Canal Street Shops,Flea Market,NaN,40.719427,-74.001901,10019,NY


In [22]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map